# Check that the Notebook Instance was Created Correctly

In [ ]:
setup_instance_check_passed = False

In [ ]:
import boto3

region = boto3.Session().region_name
session = boto3.session.Session()

ec2 = boto3.Session().client(service_name='ec2', region_name=region)
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Check Notebook Instance Name and Instance Type

In [ ]:
import json
notebook_instance_name = None

try:
    with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
        data = json.load(notebook_info)
        resource_arn = data['ResourceArn']
        region = resource_arn.split(':')[3]
        notebook_instance_name = data['ResourceName']
    print('Notebook Instance Name: {}'.format(notebook_instance_name))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: COULD NOT RETRIEVE THE NOTEBOOK INSTANCE METADATA.')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

# Check Notebook Instance Settings

In [ ]:
response = sm.describe_notebook_instance(
        NotebookInstanceName=notebook_instance_name
)
print(response)

In [ ]:
notebook_instance_arn = None

try: 
    notebook_instance_arn = response['NotebookInstanceArn']
    print('[OK] Notebook Instance ARN: {}\n'.format(notebook_instance_arn))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: Could not find the notebook instance ARN in {}.'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
direct_internet_access = None

try: 
    direct_internet_access = response['DirectInternetAccess']
    if direct_internet_access == 'Disabled':
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('[ERROR]: YOU NEED TO SET UP THE INSTANCE WITH INTERNET ACCESS ENABLED')
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    else:
        print('[OK] Direct Internet Access: {}'.format(direct_internet_access))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: CHECK YOU SET A VPC AND HAVE DIRECT INTERNET ACCESS ENABLED')
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [ ]:
min_volume_size=250

In [ ]:
volume_size = None

try:
    volume_size = response['VolumeSizeInGB']
    int(volume_size)
    if volume_size < min_volume_size: 
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
        print('[ERROR]: YOU ONLY HAVE {} GB VOLUME SIZE. YOU NEED TO SET UP YOUR INSTANCE WITH 250 GB VOLUME SIZE'.format(volume_size))
        print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    else:
        print('[OK] Volume Size: {}'.format(volume_size))
except:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: COULD NOT FIND THE VOLUME SIZE IN {}'.format(response))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

# Summary: Check All Required Settings Are Set Correctly

In [ ]:
if volume_size >= min_volume_size and direct_internet_access == 'Enabled' and notebook_instance_arn:
    setup_instance_check_passed=True
    print('[OK] Great Job!  Please Continue.')
else:
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR]: WE HAVE IDENTIFIED A MISCONFIGURATION.')
    print('[ERROR]: STOP/DELETE THIS NOTEBOOK INSTANCE AND CREATE A NEW ONE FOLLOWING THE INSTRUCTIONS')
    print('[ERROR]: CURRENT (MISCONFIGURED) SETTINGS: volume_size = {}, direct_internet_access = {}, notebook_instance_arn = {}'.format(volume_size, direct_internet_access, notebook_instance_arn))
    print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

# Please Make Sure You Don't See ^^ ERROR ^^ Message Above

# **_If You See Any ERROR Messages Above, You Must Do The Following:_** 

# **_1) Stop This Notebook Instance, 2) Wait For The Notebook Instance To Stop, 3) Delete This Notebook Instance, and 4) Re-Create This Notebook Instance._**

In [ ]:
print(setup_instance_check_passed)

In [ ]:
%store setup_instance_check_passed

In [ ]:
%store 

# Release Resources

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();